In [6]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import astropy.io.fits as pyfits
from emccd_detect.emccd_detect import EMCCDDetectBase
from PhotonCount.corr_photon_count import get_count_rate
import os
from pathlib import Path
from scipy.ndimage import rotate
from scipy.interpolate import RegularGridInterpolator
import h5py
from matplotlib.patches import Circle
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from emccd_detect.emccd_detect import EMCCDDetect, emccd_detect
from PIL import Image
from astropy.io import fits
from scipy.ndimage.interpolation import zoom
from numpy import unravel_index
from matplotlib.colors import LogNorm
from scipy.ndimage import rotate
from scipy.ndimage import zoom
from __future__ import absolute_import, division, print_function
# nav to site packages
import sys
sys.path.append('/Users/polaris/opt/anaconda3/lib/python3.9/site-packages') 
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from emccd_detect.emccd_detect import EMCCDDetect, emccd_detect
from PIL import Image
from astropy.io import fits
from scipy.ndimage.interpolation import zoom
from numpy import unravel_index

In [7]:
file_dir='/Users/polaris/Onedrive-UA/OneDrive - University of Arizona/Debris-Disk-Modelling/spc_wfov_os11/'

# Raw Emccd frames from spc os11
polx=fits.open(file_dir+'spc_wfov_os11_polx_darkhole_noiseless.fits')
poly=fits.open(file_dir+'spc_wfov_os11_poly_darkhole_noiseless.fits')
nimages=polx[0].data.shape[0]
nx=polx[0].data.shape[1]
ny=polx[0].data.shape[2]
unpol=np.zeros((nimages,ny,nx), dtype='float32')
for i in range (1830):
    unpol[i,:,:]=(polx[0].data[i,:,:]+poly[0].data[i,:,:])
input_file = unpol

In [8]:
data = np.loadtxt(file_dir+'os11_info.txt', skiprows=2)
batch = data[:,0].astype(int)
batch

array([  0, 100, 101, 102, 103,   1, 104, 105, 106, 107,   2,   3, 108,
       109, 110, 111,   4, 112, 113, 114, 115,   5])

In [9]:
# EMCCD parameters
 
full_well_serial = 100000.0
full_well = 60000.0
dark_rate = 8.33e-4
cic_noise = 0.02
read_noise = 100.0
bias = 0
cr_rate = 0                    
qe = 1.0                            # QE is already applied in input images
pixel_pitch = 13e-6               
e_per_dn = 1.0                      
nbits = 14                          # ADC bits
numel_gain_register = 604           
 # read noiseless input images as 3D array; they are sampled in time according
# to the chosen frame time for each star and are in units of e-/sec (pre-gain)
  
input_images=input_file
nimages = input_images.shape[0]
ny = input_images.shape[1]
nx = input_images.shape[2]
ccd_images=np.zeros((nimages,nx,ny),dtype='float32')
data = np.loadtxt(file_dir+'os11_info.txt', skiprows=2)
batch = data[:,0].astype(int)
start_time_h = data[:,1]
star = data[:,2].astype(int)
roll = data[:,3]
frame_exptime_sec = data[:,4]
gain = data[:,5]
nframes = data[:,6].astype(int)
nbatch = batch.shape[0]
istart = 0 
for ibatch in range(nbatch):
    print( "Processing batch "+str(batch[ibatch]) )
    iend = istart + nframes[ibatch] 

    batch_images = input_images[istart:iend,:,:]
    batch_gain = gain[ibatch]
    batch_frametime_sec = frame_exptime_sec[ibatch]

    # add EMCCD noise

    emccd = EMCCDDetectBase( em_gain=batch_gain, full_well_image=full_well, 
                            full_well_serial=full_well_serial, dark_current=dark_rate, 
                            cic=cic_noise, read_noise=read_noise, bias=bias,
                            qe=qe, cr_rate=cr_rate, pixel_pitch=pixel_pitch, 
                            eperdn=e_per_dn, numel_gain_register=numel_gain_register, nbits=nbits )

    noisy_batch_images = np.zeros(batch_images.shape, dtype='float32' )
    noisy_image = np.zeros( batch_images.shape, dtype='float32' )

    print("  batch_frametime_sec = "+str(batch_frametime_sec))
    print("  batch gain = "+str(batch_gain))
    
    for iframe in range(nframes[ibatch]):
        noisy_batch_images[iframe,:,:] = emccd.sim_sub_frame(batch_images[iframe,:,:], batch_frametime_sec).astype('float32')
        ccd_images[istart+iframe,:,:]=noisy_batch_images[iframe,:,:]
    istart = iend

Processing batch 0
  batch_frametime_sec = 14.0
  batch gain = 5000.0
Processing batch 100
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 101
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 102
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 103
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 1
  batch_frametime_sec = 14.0
  batch gain = 5000.0
Processing batch 104
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 105
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 106
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 107
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 2
  batch_frametime_sec = 14.0
  batch gain = 5000.0
Processing batch 3
  batch_frametime_sec = 14.0
  batch gain = 5000.0
Processing batch 108
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing batch 109
  batch_frametime_sec = 1.0
  batch gain = 5000.0
Processing

In [10]:
f_dir=f_dir='/Users/polaris/Onedrive-UA/OneDrive - University of Arizona/Debris-Disk-Modelling/Debris-Disk-Modelling/Eps-eri-Roman_pol/SPC_band/'
hdu = fits.PrimaryHDU(ccd_images)
hdul = fits.HDUList([hdu])
hdul.writeto(f_dir+'spc_os11_speckle_noise.fits',overwrite=True)